## Feature Selection VS Feature Extraction

#### https://www.analyticsvidhya.com/blog/2021/06/dimensionality-reduction-using-autoencoders-in-python/

Feature Selection
- Recursive Feature Elimination 
- Genetic Feature Selection 
- Sequential Forward Selection 

Feature Extraction
- AutoEncoder 
- Principal Component Analysis (PCA)
- Linear Determinant Analysis (LDA)

Types of AutoEncoders available are
- Deep Autoencoder
- Sparse Autoencoder
- Under complete Autoencoder
- Variational Autoencoder
- LSTM Autoencoer

Hyperparameters of an AutoEncoder
- Code size or the number of units in the bottleneck layer
- Input and output size, which is the number of features in the data
- Number of neurons or nodes per layer
- Number of layers in encoder and decoder
- Activation function
- Optimization function

Example

https://docs.anaconda.com/anaconda/user-guide/tasks/tensorflow/

In [1]:
#conda create -n tf-gpu tensorflow-gpu

In [4]:
!pip install pandas

In [7]:
!pip install kerastuner

ERROR: Could not find a version that satisfies the requirement kerastuner (from versions: none)
ERROR: No matching distribution found for kerastuner


In [14]:
!pip install matplotlib

In [16]:
!pip install sklearn

  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1315 sha256=59b1f344b1454f70a9704998f3d2e22158eb80b0cc7bd383694344db308b7537
  Stored in directory: c:\users\konutech\appdata\local\pip\cache\wheels\e4\7b\98\b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built sklearn


In [57]:
!pip install keras

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.5.0 requires flatbuffers~=1.12, but you have flatbuffers 20210226132247 which is incompatible.
tensorflow 2.5.0 requires grpcio~=1.34.0, but you have grpcio 1.36.1 which is incompatible.



  Using cached keras-2.7.0-py2.py3-none-any.whl (1.3 MB)


In [59]:
import math
import pandas as pd
import tensorflow as tf
#import kerastuner.tuners as kt
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredLogarithmicError
from sklearn.preprocessing import MinMaxScaler 

In [12]:
print(tf.__version__)

2.5.0


In [ ]:
#pd.options.display.float_format = '{:.1f}'.format

In [18]:
california_housing_dataframe = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",")

In [22]:
california_housing_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           17000 non-null  float64
 1   latitude            17000 non-null  float64
 2   housing_median_age  17000 non-null  float64
 3   total_rooms         17000 non-null  float64
 4   total_bedrooms      17000 non-null  float64
 5   population          17000 non-null  float64
 6   households          17000 non-null  float64
 7   median_income       17000 non-null  float64
 8   median_house_value  17000 non-null  float64
dtypes: float64(9)
memory usage: 1.2 MB


In [21]:
california_housing_dataframe.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


In [20]:
california_housing_dataframe.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000
mean,-119.562108,35.625225,28.589353,2643.664412,539.410824,1429.573941,501.221941,3.883578,207300.912353
std,2.005166,2.137340,12.586937,2179.947071,421.499452,1147.852959,384.520841,1.908157,115983.764387
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.790000,33.930000,18.000000,1462.000000,297.000000,790.000000,282.000000,2.566375,119400.000000
50%,-118.490000,34.250000,29.000000,2127.000000,434.000000,1167.000000,409.000000,3.544600,180400.000000
75%,-118.000000,37.720000,37.000000,3151.250000,648.250000,1721.000000,605.250000,4.767000,265000.000000
max,-114.310000,41.950000,52.000000,37937.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [31]:
# data in google colab
TRAIN_DATA_PATH = "https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv"
TEST_DATA_PATH = "https://download.mlcc.google.com/mledu-datasets/california_housing_test.csv"
TARGET_NAME = 'median_house_value'

In [32]:
train_data = pd.read_csv(TRAIN_DATA_PATH, sep=",")
test_data = pd.read_csv(TEST_DATA_PATH, sep=",")

x_train, y_train = train_data.drop(TARGET_NAME, axis=1), train_data[TARGET_NAME]
x_test, y_test = test_data.drop(TARGET_NAME, axis=1),  test_data[TARGET_NAME]

In [36]:
def scale_datasets(x_train, x_test):
    """
    Standard Scale test and train data
    """
    standard_scaler = MinMaxScaler()

    x_train_scaled = pd.DataFrame(
        standard_scaler.fit_transform(x_train),
        columns=x_train.columns
    )

    x_test_scaled = pd.DataFrame(
        standard_scaler.transform(x_test),
        columns = x_test.columns
    )
    
    return x_train_scaled, x_test_scaled

In [37]:
x_train_scaled, x_test_scaled = scale_datasets(x_train, x_test)

In [103]:
class AutoEncoders(Model):
    
    def __init__(self, output_units):
        super(AutoEncoders, self).__init__()
        self.encoder = Sequential(
            [
                Dense(32, activation="relu"),
                Dense(16, activation="relu"),
                Dense(3, activation="relu")
            ]
        )
        
        self.decoder = Sequential(
            [
                Dense(16, activation="relu"),
                Dense(32, activation="relu"),
                Dense(output_units, activation="sigmoid")
            ]
        )

    def call(self, inputs):
    
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
    
        return decoded

In [104]:
auto_encoder = AutoEncoders(len(x_train_scaled.columns))

auto_encoder.compile(
    loss='mae',
    metrics=['mae'],
    optimizer='adam'
)

In [105]:
history = auto_encoder.fit(
    x_train_scaled,
    x_train_scaled,
    epochs=15,
    batch_size=32,
    validation_data=(x_test_scaled, x_test_scaled)
)

Epoch 1/15
532/532 [==============================] - 4s 6ms/step - loss: 0.1012 - mae: 0.1012 - val_loss: 0.0504 - val_mae: 0.0504
Epoch 2/15
532/532 [==============================] - 3s 5ms/step - loss: 0.0461 - mae: 0.0461 - val_loss: 0.0410 - val_mae: 0.0410
Epoch 3/15
532/532 [==============================] - 3s 5ms/step - loss: 0.0395 - mae: 0.0395 - val_loss: 0.0372 - val_mae: 0.0372
Epoch 4/15
532/532 [==============================] - 3s 6ms/step - loss: 0.0369 - mae: 0.0369 - val_loss: 0.0356 - val_mae: 0.0356
Epoch 5/15
532/532 [==============================] - 3s 6ms/step - loss: 0.0354 - mae: 0.0354 - val_loss: 0.0340 - val_mae: 0.0340
Epoch 6/15
532/532 [==============================] - 3s 5ms/step - loss: 0.0348 - mae: 0.0348 - val_loss: 0.0342 - val_mae: 0.0342
Epoch 7/15
532/532 [==============================] - 3s 5ms/step - loss: 0.0345 - mae: 0.0345 - val_loss: 0.0336 - val_mae: 0.0336
Epoch 8/15
532/532 [==============================] - 3s 6ms/step - loss: 0.

In [109]:
layer_names = [layer.name for layer in auto_encoder.layers]

In [110]:
layer_names

['sequential_10', 'sequential_11']

In [111]:
encoder_layer = auto_encoder.get_layer('sequential_10')

In [112]:
reduced_df = pd.DataFrame(encoder_layer.predict(x_train_scaled))

In [113]:
reduced_df

,0,1,2
0,0.457429,1.755553,0.0
1,0.609428,1.764719,0.0
2,0.369452,2.083229,0.0
3,0.308045,1.952719,0.0
4,0.430631,2.229617,0.0
...,...,...,...
16995,3.125538,2.452579,0.0
16996,2.828446,1.819286,0.0
16997,2.700769,1.132381,0.0
16998,2.744479,1.214879,0.0
